### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Crohns_Disease/GSE66407'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = 3  # 'diagnosis' key corresponding to Crohn's Disease
age_row = 2  # 'age' key corresponding to age
gender_row = None  # Gender information is not available

def convert_trait(value):
    val = value.split(':')[-1].strip()
    if val == 'CD':
        return 1
    elif val == 'Control' or val == 'UC':
        return 0
    else:
        return None

def convert_age(value):
    if isinstance(value, str):
        val = value.split(':')[-1].strip()
    else:
        val = value
    try:
        return float(val)
    except ValueError:
        return None

def convert_gender(value):
    return None  # Gender data not available, return None for any input

save_cohort_info('GSE66407', './preprocessed/Crohns_Disease/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Crohns_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Crohns_Disease/trait_data/GSE66407.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
import re
from utils.preprocess import *

# Ensure gene_annotation is defined
cohort_dir = '../DATA/GEO/Crohns_Disease/GSE66407'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 1. Identify the keys for identifier and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Description'  # Extract gene symbol from the string, e.g., "tetraspanin 6 [Source:HGNC Symbol;Acc:11858]"

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Extract gene symbols from the 'Description' field
mapping_df['Gene'] = mapping_df[gene_symbol_key].apply(lambda x: re.search(r'\w+', x).group() if pd.notnull(x) else None)
mapping_df = mapping_df.dropna(subset=['Gene', identifier_key])

# Ensure gene_data is defined
gene_data = get_genetic_data(matrix_file)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library.
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
from utils.preprocess import *

# Ensure the matrix_file path from the previous steps
cohort_dir = '../DATA/GEO/Crohns_Disease/GSE66407'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# Ensure gene_annotation is available
gene_annotation = get_gene_annotation(soft_file)

# Extract gene symbols from the 'Description' field
import re

identifier_key = 'ID'
gene_symbol_key = 'Description'  # Extract gene symbol from the string, e.g., "tetraspanin 6 [Source:HGNC Symbol;Acc:11858]"

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Extract gene symbols from the 'Description' field
mapping_df['Gene'] = mapping_df[gene_symbol_key].apply(lambda x: re.search(r'\w+', x).group() if pd.notnull(x) else None)
mapping_df = mapping_df.dropna(subset=['Gene', identifier_key])

# Ensure gene_data is correctly defined
gene_data = get_genetic_data(matrix_file)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Crohns_Disease/gene_data/GSE66407.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
csv_path = './preprocessed/Crohns_Disease/trait_data/GSE66407.csv'
selected_clinical_data = pd.read_csv(csv_path, index_col=0)
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Crohns_Disease')

# 4. Save the cohort information.
save_cohort_info('GSE66407', './preprocessed/Crohns_Disease/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    merged_data_csv_path = './preprocessed/Crohns_Disease/GSE66407.csv'
    unbiased_merged_data.to_csv(merged_data_csv_path)
